In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
#r ".\binaries\ApplicationWithIDT.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using ApplicationWithIDT;
using XESF;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


# Run Simulation

In [2]:
var date = DateTime.Now.ToString("ddMMyyyy");
date

01092023

open a database

In [3]:
string dbPath = @"WedgeFlow_db_" + date ;
var database = OpenOrCreateDatabase(dbPath);

Creating database 'WedgeFlow_db_01092023'.


run WedgeFlow simulation and safe to database

In [4]:
BoSSS.Solution.Application.InitMPI();
BoSSS.Solution.Application.DeleteOldPlotFiles();
var p = new XESFMain();

var C = XESFHardCodedControl.XDGWedgeFlow_TwoLs_Base(
    dbPath:dbPath,
    optiLSDegree: 1,
    lsDegree: 1,
    shocksetup: ApplicationWithIDT.GetLevelSet.FromFunction,
    optiLevelSetType: OptiLevelSetType.SplineLevelSet,
    initialValue: ApplicationWithIDT.GetInitialValue.FromFunctionPerSpecies,
    MaxIterations: 200,
    dgDegree: 0,
    numOfCellsX: 15,
    numOfCellsY: 10,
    initialAngle_shockLS: 32,
    PlotInterval: -1,
    interfaceFluxLS2: XESF.Fluxes.ConvectiveInterfaceFluxes.GodunovInterface,
    bulkFlux: XESF.Fluxes.ConvectiveBulkFluxes.OptimizedHLLC,
    agg: 0.4
);
p.Init(C);
p.RunSolverMode();


rm;
Grid Edge Tags changed.
Session ID: 2f00b49c-544d-4179-a231-1a674f60fb21, DB path: 'WedgeFlow_db_01092023'
Session directory 'WedgeFlow_db_01092023\sessions\2f00b49c-544d-4179-a231-1a674f60fb21'.
Initial Value : l2: |r|= 1.0778378E+01, |f_err|=2.4570273E+01, | f_phi |=0E+00,

Starting Opt-Iter No.1: 
alpha, ||R1||  1, Error 
alpha, ||R1||  0.5, Error 
alpha, ||R1||  0.25, Error 
alpha, M, predM  0.125, 10770.664450276212, 12304.914127643633
l2: ||R0||= 9.0972954E+00, f_err=2.150577E+01, f_phi=0E+00, Alpha = 1.25E-01, Gamma = 1E+00, Kappa = 1E+03, Mu = 1.63E+02, QuadOrder=2
**NOT** terminating now, minimal number of iterations not reached


Starting Opt-Iter No.2: 
alpha, ||R1|| 1 LevelSetCFL 
alpha, ||R1|| 0.5 LevelSetCFL 
alpha, ||R1||  0.25, Error 
alpha, ||R1||  0.125, Error 
alpha, ||R1||  0.0625, Error 
alpha, M, predM  0.03125, 5648.870862465331, 5835.952721422451
l2: ||R0||= 8.8168796E+00, f_err=2.098005E+01, f_phi=0E+00, Alpha = 3.13E-02, Gamma = 1E+00, Kappa = 1E+03, Mu = 

# Postprocessing

we need to load something from the ApplicationWithIDT assembly 

In [5]:
Assembler aaa = new Assembler(3);
var si = database.Sessions.Pick(0);

## Enthalpy error

enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(c_P -1)(\rho E - k),~k=c_P \text{Ma}^2\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$

For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

As we have uniform inflow the v-velocity is zero and we have
$$ h_{in} = \frac{\gamma}{\gamma-1} + \frac{1}{2}u_{in}^2 $$
We prescribe the inflow conditions in terms of pressure $p_{in}=1$, density $\rho_{in}=1$ and the Mach number $M_{in}=2$. From those the velocity is computed as 
$$ u_{in} = M_{in} \sqrt{\gamma \frac{p_{in}}{\rho_{in}}}= 2\sqrt{\gamma}$$

In [6]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 2;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

6.300000000000001

In [7]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

In [8]:
p.GetResEnthalpyPlot(si,EEN).ToGnuplot().PlotSVG(xRes:1200,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -14 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 10 2 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 40 
 
 
 
 
 45 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||R(z)|| 
 
 
 ||R(z)|| 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M988.5,57.1 L1041.9,57.1 M53.9,51.6 L78.3,53.0 L102.6,53.3 L127.0,53.6 L151.3,55.2 L175.7,56.7
 L200.1,57.8 L224.4,59.3 L248.8,68.6 L273.1,70.2 L297.5,70.8 L321.9,71.6 L346.2,73.6 L370.6,80.0
 L394.9,81.8 L419.3,85.9 L443.7,93.8 L468.0,112.7 L492.4,147.8 L516.7,211.4 L541.1,253.8 L565.5,264.0
 L589.8,275.9 L614.2,289.6 L638.5,292.7 L662.9,293.2 L687.3,293.0 L711.6,293.4 L736.0,293.5 L760.3,293.5
 L784.7,293.5 L809.1,293.5 L833.4,293.5 L857.8,293.5 L882.1,293.5 L906.5,293.5 L930.9,293.5 L955.2,293.5
 L979.6,293.5 L1003.9,293.5 L1028.3,293.5 L1052.7,293.5 L1077.0,293.5 L1101.4,293.5 L1125.7,293.5 L1150.1,293.5
 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r(z)|| 
 
 
 ||r(z)|| 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M988.5,81.1 L1041.9,81.1 M53.9,60.7 L78.3,62.5 L102.6,62.9 L127.0,63.2 L151.3,65.5 L175.7,67.1
 L200.1,68.6 L224.4,70.0 L248.8,92.4 L273.1,93.5 L297.5,90.6 L321.9,90.4 L346.2,91.7 L370.6,97.9
 L394.9,98.3 L419.3,107.9 L443.7,105.4 L468.0,125.1 L492.4,160.1 L516.7,226.5 L541.1,271.2 L565.5,277.6
 L589.8,288.0 L614.2,305.0 L638.5,308.2 L662.9,320.3 L687.3,316.6 L711.6,329.4 L736.0,347.0 L760.3,369.0
 L784.7,383.9 L809.1,394.3 L833.4,403.6 L857.8,410.0 L882.1,415.7 L906.5,420.1 L930.9,421.4 L955.2,421.8
 L979.6,424.1 L1003.9,424.1 L1028.3,424.1 L1052.7,424.1 L1077.0,424.0 L1101.4,423.9 L1125.7,424.0 L1150.1,424.0
 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||herr|| 
 
 
 ||h err || 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M988.5,105.1 L1041.9,105.1 M53.9,174.1 L78.3,174.6 L102.6,173.9 L127.0,173.1 L151.3,164.6 L175.7,163.9
 L200.1,162.6 L224.4,162.3 L248.8,153.7 L273.1,153.7 L297.5,154.9 L321.9,158.4 L346.2,162.4 L370.6,168.2
 L394.9,171.5 L419.3,173.0 L443.7,197.1 L468.0,247.1 L492.4,309.2 L516.7,353.1 L541.1,337.4 L565.5,349.4
 L589.8,358.9 L614.2,367.7 L638.5,369.6 L662.9,369.0 L687.3,369.0 L711.6,368.7 L736.0,368.5 L760.3,368.5
 L784.7,368.5 L809.1,368.5 L833.4,368.5 L857.8,368.5 L882.1,368.5 L906.5,368.5 L930.9,368.5 L955.2,368.5
 L979.6,368.5 L1003.9,368.5 L1028.3,368.5 L1052.7,368.5 L1077.0,368.5 L1101.4,368.5 L1125.7,368.5 L1150.1,368.5
 '/>

In [9]:
p.PlotShadowFields(si);